In [1]:
from pathlib import Path

import pysindy as ps
from exp.expdata import ExpData, LorenzExp
from exp.metrics import coeff_metrics, data_metrics
import jax.numpy as jnp
import jax.numpy as jnp
import numpy as np
import pickle
import time

In [2]:
initial_state = jnp.array([ 0.37719066, -0.39631459, 16.92126795])
true_sigma2 = 0.0
dt = 0.02
n_colloc = None
expdata = LorenzExp(
    initial_state=initial_state,
    t0=0.0,
    t1=10.01,
    dt = 0.01,
    dt_train=dt,
    noise= jnp.sqrt(true_sigma2),
    seed=32,
    n_colloc=n_colloc,
    one_rkey=True,
    feature_names=['x','y','z']
)

poly_lib = ps.PolynomialLibrary(degree=2)
base_opt = ps.STLSQ(threshold=0.25, alpha=0.)
optimizer = ps.optimizers.EnsembleOptimizer(
    opt = base_opt,
    library_ensemble = True,
    n_models = 2000,
    n_subset = int(0.8*expdata.t_train.shape[0]),
    replace=True,
    ensemble_aggregator = lambda x: np.mean(x,axis=0)
)


model = ps.SINDy(
    feature_library=poly_lib, 
    optimizer=optimizer,
    feature_names=['x','y','z'],
    differentiation_method=ps.SmoothedFiniteDifference()
)

model.fit(
    x = np.array(expdata.x_train),
    t = np.array(expdata.t_train),
)


ValueError: solve: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (m,m),(m,n)->(m,n) (size 1 is different from 3)

In [3]:
expdata.x_train.shape, expdata.t_train.shape

((501, 3), (501,))